# 4.FE
Gồm có 2 bước chính
- Feature Generation: ở bước này tập trung các kĩ thuật để tạo ra feature: xử lí nlp, image, binning, scaling, grouping, aggregate.
  - Kết hợp với Modelling để tìm ra nhóm feature tốt và tập trung mạnh vào đó
  
- Feature Selection: ở bước này tập trung các kĩ thuật chọn feature để tốt ưu mô hình
  - Chọn được feature tốt và tối ưu được bộ nhớ
  - Chọn được model parameters tốt nhất phù hợp với bộ features tốt nhất
  - Kết hợp với các kĩ thuật chia dataset (CV, leave-one)




## 4.1.Feature Generation

[Simple FE](https://machinelearningcoban.com/general/2017/02/06/featureengineering/)
-    Trực tiếp lấy raw data
-    Bag-of-words
  -    Bag-of-Words trong Computer Vision
-    Feature Scaling and Normalization
  -        Rescaling
  -        Standardization
  -        Scaling to unit length


## 4.2.Feature Selection

- Tiêu chí đánh giá metric of feature
  - AUC: thể hiện khả năng dự đoán
  - Correlation: kiểm tra độ tương quan giữa feature với target hoặc với các feature quan trọng khác
  - Converage: kiểm tra null, null nhiều thì ít thông tin
  - Weighted of evidence, and information value:
    - $\ln \left(\frac{P(\text {Good})}{P(\text {Bad})}\right)$
    - \begin{array}{l}{\text { Information value }} \\ {\qquad \sum(P(G o o d)-P(B a d)) * \ln \left(\frac{P(G o d)}{P(B a d)}\right)}\end{array}



[Brute Force Approach](https://www.kdnuggets.com/2017/11/rapidminer-basic-concepts-feature-selection.html)
Cách tiếp cận bằng các chạy thử thất cả các feature combination và so sánh trên các metrics of feature

[Feature Selection](https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/) 
- Forwarding: ta bắt đầu với tập feature rỗng, sau đó lần lượt add thêm feature vào tập này. Nếu thấy performance của model tăng ta sẽ tiếp tục quá trình này, ngược lại sẽ dừng lại.
- Backwarding: ta bắt đầu với toàn bộ tập feature, sau đó lần lượt remove từng feature khỏi tập này. Nếu thấy performance của model tăng hoặc giảm không quá nhiều, ta sẽ tiếp tục quá trình này, ngược lại nếu performance bị drop quá mạnh sẽ dừng lại.
- Hybridge: kết hợp cả 2 hướng trên


Research:
- [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)
- [2](https://www.kaggle.com/sz8416/6-ways-for-feature-selection) example selectkbes-RFE in kaggle
- [3](https://www.kaggle.com/dkim1992/feature-selection-ranking) chua doc
- [4](https://machinelearningmastery.com/feature-selection-machine-learning-python/) tong quat
- [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html)
- [5](https://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/) chua doc

**RFE**

[theory](https://www.kaggle.com/nroman/recursive-feature-elimination)

Trong mini course này, tôi sẽ áp dụng hướng “backwarding”. Các bước thực hiện như sau:

- Đặt: n là số lần lặp feature selection, k là số feature sẽ drop ở mỗi lần lặp, p là AUC sau mỗi lần train
Train model với XGboost
- Lấy kết quả feature important sắp xếp giảm dần và loại ra k feature có giá trị thấp nhất
- Lưu lại performance hiện tại để so sánh với performance tiếp theo.
- Nếu thấp hơn ngưỡng p sẽ dừng
- Tiếp tục quá trình selection

Tuỳ theo số lượng feature và cài đặt hyper-parameter của model thì thời gian sẽ nhanh chậm khác nhau.

Additional:
- Genetic algorithm for feature selection

Source:
- [1](https://ongxuanhong.wordpress.com/2019/04/17/data-science-mini-course/#more-15645)
- [2](https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15) doc them

---
**Boruta**
[1](http://danielhomola.com/2015/05/08/borutapy-an-all-relevant-feature-selection-method/)

## Genetic algorithm
- [Explain 1](https://www.neuraldesigner.com/blog/genetic_algorithms_for_feature_selection)
- [Explain 2](https://towardsdatascience.com/feature-reduction-using-genetic-algorithm-with-python-403a5f4ef0c1)
- [Explain 3](http://dkopczyk.quantee.co.uk/genetic-algorithm/)

In [0]:
import os
import random
import numpy as np
import pandas as pd
import shutil

from sklearn.svm import SVR
from joblib import Parallel, delayed

# load train and valid
data_folder = 'competitionData'
train = pd.read_csv(os.path.join(data_folder, 'trainingDataDeflated.csv'))
valid = pd.read_csv(os.path.join(data_folder, 'validationDataDeflated.csv'))

# get training columns
training_columns = list(train.columns.values)
for column in ['ind', 'deck', 'nofGames', 'nOfPlayers', 'winRate']:
    training_columns.remove(column)
print("Total number of training columns: {}".format(len(training_columns)))

# set index for loc
train.set_index('ind', inplace=True)
valid.set_index('ind', inplace=True)

def load_dna_from_file(filename):
    dna = []
    f = open(filename)
    while True:
        line = f.readline().strip()
        if line == '':
            break
        # parse params
        params = line.split(";")
        indices = [int(item) for item in params[3].split(',') if item != '']
        # add information
        gene = dict()
        gene["epsilon"] = float(params[0])
        gene["c"] = float(params[1])
        gene["gamma"] = round(float(params[2]), 5)
        gene["indices"] = indices
        dna.append(gene)
    f.close()
    return dna

'''
save dna to file
'''
def save_dna_to_file(item):
    dna = item[0]
    fitness_score = item[1]
    filename = os.path.join("finalPopulation", "{}.txt".format(fitness_score))

    fout = open(filename, "w")
    for gene in dna:
        ind_text = ','.join(str(index) for index in gene["indices"])
        text = ';'.join([str(gene["epsilon"]), str(gene["c"]), str(gene["gamma"]), ind_text])
        fout.write(text + '\n')

    fout.close()

'''
initialize the first population, either randomly or from a file
'''
def init_population(population_size, from_files=False):
    pop = []

    # add good dna to the init population from previously tested good solutions
    if from_files == True:
        files = os.listdir("goodSolutions")
        for file in files:
            fitness_score = float(file[:-4])
            filename = os.path.join("goodSolutions", file)
            dna = load_dna_from_file(filename)
            pop.append([dna, fitness_score])

    num_of_random_dna = population_size - len(pop)
    for i in range(num_of_random_dna):
        dna = []
        for num_of_data in range(600, 1501, 100):
            gene = dict()
            gene["c"] = 5.0
            gene["gamma"] = 1.0/90
            gene["epsilon"] = 0.02
            gene["indices"] = sorted(random.sample(range(1, 100001), num_of_data))
            dna.append(gene)
        pop.append([dna, 0.0])

    min_score = min([item[1] for item in pop])
    return pop, min_score

'''
R2 predict evaluation
'''
def R2(x, y):
    return 1 - np.sum(np.square(x - y)) / np.sum(np.square(y - np.mean(y)))

'''
evaluate scores of a dna, sequential processing
'''
def fitness_sequential(dna, weight_train = 0.5):

    gene_valid_r2 = []
    gene_train_r2 = []

    for gene in dna:
        c = gene["c"]
        gamma = gene["gamma"]
        epsilon = gene["epsilon"]
        indices = gene["indices"]

        svr = SVR(kernel='rbf', gamma=gamma, C=c, epsilon=epsilon, shrinking=False)
        svr.fit(train.loc[indices][training_columns], train.loc[indices]["winRate"])
        valid_pred = svr.predict(valid[training_columns])
        valid_r2 = R2(valid_pred, valid["winRate"])
        gene_valid_r2.append(valid_r2)

        unselected_indices = list(set(range(1, 100001)) - set(indices))
        train_pred = svr.predict(train.loc[unselected_indices][training_columns])
        train_r2 = R2(train_pred, train.loc[unselected_indices]["winRate"])
        gene_train_r2.append(train_r2)

    fitness_valid = np.mean(gene_valid_r2)
    fitness_train = np.mean(gene_train_r2)
    fitness_score = (fitness_valid + weight_train * fitness_train) / (weight_train + 1.0)
    print("{},{},{}:{}".format(fitness_score, fitness_valid, fitness_train, gene_valid_r2 + gene_train_r2))
    return fitness_score

def train_and_valid(gene, using_train = False):
    c = gene["c"]
    gamma = gene["gamma"]
    epsilon = gene["epsilon"]
    indices = gene["indices"]

    svr = SVR(kernel='rbf', gamma=gamma, C=c, epsilon=epsilon, shrinking=False)
    svr.fit(train.loc[indices][training_columns], train.loc[indices]["winRate"])
    valid_pred = svr.predict(valid[training_columns])
    valid_r2 = R2(valid_pred, valid['winRate'])

    if using_train:
        unselected_indices = list(set(range(1, 100001)) - set(indices))
        train_pred = svr.predict(train.loc[unselected_indices][training_columns])
        train_r2 = R2(train_pred, train.loc[unselected_indices]["winRate"])
        return [valid_r2, train_r2]

    else:
        return valid_r2

'''
evaluate scores of a dna, parallel processing
'''
def fitness_parallel(dna, using_train = False, weight_train = 1.0 / 16, valid_threshold = 21.0):
    gene_r2 = Parallel(n_jobs=5, verbose=1)(delayed(train_and_valid)(gene, using_train) for gene in dna)
    if using_train:
        gene_valid_r2 = [item[0] for item in gene_r2]
        gene_train_r2 = [item[1] for item in gene_r2]
        fitness_valid = np.mean(gene_valid_r2)
        fitness_train = np.mean(gene_train_r2)
        fitness_score = (fitness_valid + fitness_train * weight_train) / (weight_train + 1.0)
        print("{},{},{}:{}".format(fitness_score, fitness_valid, fitness_train, gene_valid_r2 + gene_train_r2))
    else:
        fitness_score = np.mean(gene_r2)
        print("{}:{}".format(fitness_score, gene_r2))
    return fitness_score

'''
randomly replace a number of indices
no action is taken on epsilon, gamma, or c
'''
def mutate_gene(gene, percent_indices = 10):
    num_of_indices = len(gene["indices"])
    selected_mutation_part = random.randint(0, num_of_indices * 4)
    if selected_mutation_part < num_of_indices:
        # c varies from 0.00001 to 10.0
        gene["c"] = random.randint(1, 1000000) / 100000.0
    elif selected_mutation_part < num_of_indices * 2:
        # gamma varies from 0.01 to 10.0
        gene["gamma"] = round(random.randint(1, 9000000) / 900000.0, 5)
    elif selected_mutation_part <= num_of_indices * 3:
        # epsilon varies from 0.00001 to 1.0
        gene["epsilon"] = random.randint(1, 100000) / 100000.0
    else:
        indices = gene["indices"]
        num_of_removed_indices = int(percent_indices * num_of_indices / 100)
        removed_indices = random.sample(indices, num_of_removed_indices)

        remaining_indices = set(indices) - set(removed_indices)
        pool_indices = list(set(range(1, 100001)) - remaining_indices)
        new_indices = random.sample(pool_indices, num_of_removed_indices)

        gene["indices"] = list(remaining_indices) + new_indices
        assert len(gene["indices"]) == num_of_indices, "Missing indices in mutation"

'''
randomly select a gene for mutation
'''
def mutate_dna(dna, single_gene=True):
    if single_gene:
        selected_gene = random.randint(0, 9)
        mutate_gene(dna[selected_gene])
    else:
        for selected_gene in range(10):
            mutate_gene(dna[selected_gene])
    return dna

'''
perform crossover between 2 genes
'''
def crossover_gene(gene_x, gene_y):
    new_gene = dict()
    new_gene["c"] = random.sample([gene_x["c"], gene_y["c"]], 1)[0]
    new_gene["epsilon"] = random.sample([gene_x["epsilon"], gene_y["epsilon"]], 1)[0]
    new_gene["gamma"] = random.sample([gene_x["gamma"], gene_y["gamma"]], 1)[0]

    x_indices = set(gene_x["indices"])
    y_indices = set(gene_y["indices"])
    overlapping_indices = x_indices.intersection(y_indices)

    num_of_indices = len(x_indices)
    num_of_remaining_x_indices = int((num_of_indices - len(overlapping_indices)) / 2)
    num_of_remaining_y_indices = num_of_indices - len(overlapping_indices) - num_of_remaining_x_indices

    remaining_x_indices = list(x_indices - overlapping_indices)
    remaining_y_indices = list(y_indices - overlapping_indices)

    selected_x_indices = random.sample(remaining_x_indices, num_of_remaining_x_indices)
    selected_y_indices = random.sample(remaining_y_indices, num_of_remaining_y_indices)

    new_gene["indices"] = list(overlapping_indices) + selected_x_indices + selected_y_indices
    assert len(new_gene["indices"]) == num_of_indices, "Missing indices in crossover"

    return new_gene

'''
randomly select a gene and perform crossover on that gene
'''
def crossover_dna(dna_x, dna_y):

    # perform crossover on all genes
    new_dna = []
    for i in range(10):
        gene_x = dna_x[i]
        gene_y = dna_y[i]
        new_dna.append(crossover_gene(gene_x, gene_y))

    return new_dna

'''
select dna for new generation
'''
def select(population, population_size):
    if len(population) < population_size:
        return population

    prob_distribution = [item[1] for item in population]
    selected_indices = np.random.choice(range(len(population)), population_size, prob_distribution)
    removed_indices = [index for index in range(len(population)) if index not in selected_indices]

    # set new population based on selected indices
    new_population = []
    for index in selected_indices:
        new_population.append(population[index])

    # remove unselected_indices
    for index in sorted(removed_indices, reverse=True):
        del population[index]
    del population

    return new_population

'''
clone a gene
'''
def copy_gene(gene):
    new_gene = dict()
    new_gene["c"] = gene["c"]
    new_gene["gamma"] = gene["gamma"]
    new_gene["epsilon"] = gene["epsilon"]
    new_gene["indices"] = list(gene["indices"])
    return new_gene

'''
Clone a dna
'''
def copy_dna(dna):
    new_dna = []
    for gene in dna:
        new_gene = copy_gene(gene)
        new_dna.append(new_gene)
    return new_dna

'''
Select and update files for initial generation
'''
def update_files(population_size):
    new_solutions = os.listdir("finalPopulation")
    if len(new_solutions) == 0:
        return

    for file in new_solutions:
        if not os.path.exists(os.path.join("goodSolutions", file)):
            shutil.copy2(os.path.join("finalPopulation", file), "goodSolutions")
        os.remove(os.path.join("finalPopulation", file))

    all_solutions = os.listdir("goodSolutions")
    all_scores = [float(item[:-4]) for item in all_solutions]
    top_scores = sorted(all_scores, reverse=True)[:population_size]

    for item in all_solutions:
        if float(item[:-4]) not in top_scores:
            os.remove(os.path.join("goodSolutions", item))

if __name__ == "__main__":
    counter = 1
    while True:
        POPULATION_MAX = 21
        GENERATION_SIZE = 5

        # init population
        # tao ra population ban dau
        population, MIN_LOG_SCORE = init_population(POPULATION_MAX, from_files=True)
        print("Running times {}, min log score {}".format(counter, MIN_LOG_SCORE))

        # evolve
        for i in range(GENERATION_SIZE):
            new_solutions = []
            population_size = len(population)
            num_of_selected_parents = 15 # 3 * (population_size // 6)
            print("Generation {}, input size {}, num of selected parents {}".format(i + 1, population_size, num_of_selected_parents))

            # generate dna for mutation and crossover
            selected_indices = random.sample(range(population_size), num_of_selected_parents)

            for j in range(0, num_of_selected_parents, 3):
                # generate new child from mutation
                mutated_child = mutate_dna(copy_dna(population[selected_indices[j]][0]))
                new_solutions.append(mutated_child)

                mutated_child = mutate_dna(copy_dna(population[selected_indices[j + 1]][0]))
                new_solutions.append(mutated_child)

                mutated_child = mutate_dna(copy_dna(population[selected_indices[j + 2]][0]))
                new_solutions.append(mutated_child)

                # generate new children from crossover
                crossover_child = crossover_dna(population[selected_indices[j+1]][0], population[selected_indices[j+2]][0])
                new_solutions.append(crossover_child)

            # evaluate new children
            for dna in new_solutions:
                #fitness_score = fitness_sequential(dna)
                fitness_score = fitness_parallel(dna, using_train=True)
                if fitness_score > MIN_LOG_SCORE:
                    save_dna_to_file([dna, fitness_score])
                population.append([dna, fitness_score])

            # randomly select individuals from new population for the next generation
            population = select(population, POPULATION_MAX)

        # output final population
        '''
        for item in population:
            save_dna_to_file(item)
        '''

        # update good solutions for a new round
        update_files(POPULATION_MAX)
        counter += 1

